Use with `vscode` and its `Data Wrangler`

In [1]:
import re
import numpy as np
import torch
import pickle

_GLOBAL = {}
def load_pt(file):
    data = torch.load(file)
    return data

def load_pkl(file):
    with open(file, 'rb') as f:
        data = pickle.load(f)
    return data

def load_np(file):
    return np.load(file, allow_pickle=True)
    
def load(file):
    if file.endswith('.pt'):
        data = load_pt(file)
    elif file.endswith('.pkl'):
        data = load_pkl(file)
    elif file.endswith('.npy') or file.endswith('.npz'):
        data = load_np(file)
    return data

def expand_dict(data,prefix='',depth=0, max_keys=1000):
    indent = '    ' * depth
    def _print(*args, **kwargs):print(indent[1:], *args, **kwargs)
    if depth > 2:
        _print(f'too deep, depth: {depth}')
        return
    _print(f'• LEN={len(data)}, DEPTH={depth}')
    depth += 1

    if isinstance(data, list):
        for i,x in enumerate(data):
            expand_dict(x, f'{prefix}列{i}一',depth)
    elif isinstance(data,dict):
        for k, v in data.items():
            # 如果k有非下划线符号，全部替换成_
            k = re.sub(r'[^a-zA-Z0-9_]', '_', k)

            str_v = ''
            if isinstance(v,np.ndarray):
                str_v = f'{v.shape}\t{v.dtype}'
            elif not isinstance(v, dict):
                str_v = str(v)
            _print(f"{k}:\t{str(type(v))[8:-2]}\t\t{str_v}")

            if isinstance(v, dict):
                expand_dict(v,f'{prefix}{k}一',depth)
            else:
                if isinstance(v, torch.Tensor) and v.size(0) == 1:
                    _print(f'🧽 squeezed: {prefix}{k}')
                    v = v.squeeze(0)
                elif isinstance(v, np.ndarray):
                    ...
                if len(_GLOBAL.keys()) >= max_keys:
                    _print(f'too many keys ,keys.len: {len(_GLOBAL.keys())}')
                    return
                _GLOBAL[f'{prefix}{k}'] = v

def get_global(data, max_keys=50):
    expand_dict(data, max_keys=max_keys)
    globals().update(_GLOBAL)


In [ ]:
_file = '/home/n/document/code/mocap/output/0_input_video/0_input_video.mp4_hand00.pt'
get_global(load(_file))
print(list(_GLOBAL.keys()))

 • LEN=2, DEPTH=0
    • LEN=3, DEPTH=1
    hand_bbox:	list		[320.0, 392.0, 351.0, 433.0]
    is_right:	float		1.0
    wilor_preds:	dict		
        • LEN=9, DEPTH=2
        global_orient:	numpy.ndarray		(1, 1, 3)	float32
        hand_pose:	numpy.ndarray		(1, 15, 3)	float32
        betas:	numpy.ndarray		(1, 10)	float32
        pred_cam:	numpy.ndarray		(1, 3)	float32
        pred_keypoints_3d:	numpy.ndarray		(1, 21, 3)	float32
        pred_vertices:	numpy.ndarray		(1, 778, 3)	float32
        pred_cam_t_full:	numpy.ndarray		(1, 3)	float64
        scaled_focal_length:	numpy.float64		15859.375
        pred_keypoints_2d:	numpy.ndarray		(1, 21, 2)	float64
    • LEN=3, DEPTH=1
    hand_bbox:	list		[437.0, 360.0, 464.0, 402.0]
    is_right:	float		0.0
    wilor_preds:	dict		
        • LEN=9, DEPTH=2
        global_orient:	numpy.ndarray		(1, 1, 3)	float32
        hand_pose:	numpy.ndarray		(1, 15, 3)	float32
        betas:	numpy.ndarray		(1, 10)	float32
        pred_cam:	numpy.ndarray		(1, 3)	float